This example mirrors [layout.html](https://niivue.com/demos/features/layout.html).

In [ ]:
from pathlib import Path
from ipyniivue import download_dataset

BASE_API_URL = "https://niivue.com/demos/images/"
DATA_FOLDER = Path("images")

# Download data for example
download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "pcasl.nii.gz",
    ],
)

In [ ]:
import ipywidgets as widgets

from ipyniivue import MultiplanarType, NiiVue, ShowRender

# Initialize NiiVue
nv = NiiVue()
nv.opts.show_3d_crosshair = True
nv.opts.back_color = [0.64, 0.76, 0.68, 1]
nv.opts.log_level = "debug"
nv.set_multiplanar_pad_pixels(5)
nv.graph.opacity = 1.0

# Load volume
nv.load_volumes([{"path": DATA_FOLDER / "pcasl.nii.gz"}])

# --- Controls ---

# 1. Smooth (interpolation)
interp_check = widgets.Checkbox(value=True, description="Smooth")


def on_interp_change(change):
    """Handle interpolation change."""
    nv.set_interpolation(not change.new)


interp_check.observe(on_interp_change, names="value")

# 2. Resolution (DPI)
dpi_select = widgets.Dropdown(
    options=[
        ("Enable High DPI (best)", 1),
        ("Disable High DPI", 0),
        ("DPR = 0.7", 0.7),
        ("DPR = 0.5", 0.5),
        ("DPR = 0.3 (worst)", 0.3),
    ],
    value=1,
    description="Resolution",
)


def on_dpi_change(change):
    """Handle dpi change."""
    val = change.new
    if val == 0:
        nv.set_high_resolution_capable(-1)
    elif val == 1:
        nv.set_high_resolution_capable(0)
    else:
        nv.set_high_resolution_capable(val)


dpi_select.observe(on_dpi_change, names="value")

# 3. Volume render mode
render_mode = widgets.Dropdown(
    options=[
        ("slices", -1),
        ("matte", 0),
        ("low", 0.3),
        ("medium", 0.6),
        ("high", 1.0),
    ],
    value=0,
    description="Volume Render Mode",
)


def on_render_mode_change(change):
    """Handle render mode change."""
    nv.set_volume_render_illumination(float(change.new))


render_mode.observe(on_render_mode_change, names="value")

# 4. Timeline
timeline_check = widgets.Checkbox(value=False, description="Timeline")


def on_timeline_change(change):
    """Handle timeline checkbox change."""
    nv.graph.auto_size_multiplanar = change.new


timeline_check.observe(on_timeline_change, names="value")

# 5. Padding
pad_slider = widgets.IntSlider(min=0, max=10, value=5, description="Padding")


def on_pad_change(change):
    """Handle pad slider change."""
    nv.set_multiplanar_pad_pixels(change.new)


pad_slider.observe(on_pad_change, names="value")

# 6. Layout
layout_select = widgets.Dropdown(
    options=[("Auto", 0), ("Column", 1), ("Grid", 2), ("Row", 3)],
    value=0,
    description="Layout",
)


def on_layout_change(change):
    """Handle layout change."""
    nv.opts.multiplanar_layout = MultiplanarType(change.new)


layout_select.observe(on_layout_change, names="value")

# 7. Rendering
rendering_select = widgets.Dropdown(
    options=[("Never", 0), ("Always", 1), ("Auto", 2)], value=2, description="Rendering"
)


def on_rendering_change(change):
    """Handle rendering change."""
    nv.opts.multiplanar_show_render = ShowRender(change.new)
    nv.draw_scene()


rendering_select.observe(on_rendering_change, names="value")

# 8. Hero image
hero_slider = widgets.IntSlider(min=0, max=9, value=0, description="Hero Image")


def on_hero_change(change):
    """Handle hero slider change."""
    nv.set_hero_image(change.new * 0.1)


hero_slider.observe(on_hero_change, names="value")

# Display all
ui = widgets.VBox(
    [
        widgets.HBox([interp_check, dpi_select]),
        widgets.HBox([render_mode, timeline_check]),
        widgets.HBox([pad_slider, layout_select]),
        widgets.HBox([rendering_select, hero_slider]),
        nv,
    ]
)

ui